# Implementing C3D Model for Video Classification

In [1]:
%load_ext autoreload
%autoreload 2

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import sys
import os
import time

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

 
# adding video-download folder to the system path
sys.path.insert(0, '/workspace/youtube-humpback-whale-classifier/video-download')
 
# importing read_frames_hdf5 function
from hdf5_data_loading import read_frames_hdf5

#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [2]:
import wandb

#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

wandb.run.name = "resnet-stacked-bidirectional-lstm"

wandb: Currently logged in as: micheller (epg). Use `wandb login --relogin` to force relogin


In [3]:
print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU'))) #1 if we select GPU mode in Colab Notebook, 0 if running on local machine

Num GPUs available:  4


## Load Dataset

In [4]:
#load dataset in
data = pd.read_csv(workspace_path + '/downloaded_videos.csv')
y = data.pop('relevant')
X = data

## Load Frames

In [5]:
#load in frames for all videos
start = time.time()

N = X.shape[0] #number of videos in our dataset
videos = np.empty((N, 461, 224, 224, 3), dtype=np.float32)
labels = np.empty(N, dtype = np.uint8)

for i, video in enumerate(list(X.renamed_title)):
    if i % 50 == 0:
        print(f'Loading frames for video {i}...')
        
    clip_name = video.replace("_", "_clip_").replace(".mp4", "")
    frames, frame_labels = read_frames_hdf5(clip_name) #returns frames array of shape (461, 224, 224, 3)
    
    videos[i, ...] = frames
    labels[i] = frame_labels[0] #all frames have the same label since label is assigned to overall video

stop = time.time()
print(f'Done loading frames in {stop-start} seconds.')
videos.shape

Loading frames for video 0...
Loading frames for video 50...
Loading frames for video 100...
Loading frames for video 150...
Loading frames for video 200...
Loading frames for video 250...
Loading frames for video 300...
Loading frames for video 350...
Done loading frames in 582.3002216815948 seconds.


(364, 461, 224, 224, 3)

## Get Features

In [6]:
from cnn import CNN

with tf.device('/CPU:0'):
    ConvNet = CNN(224)
    feature_extractor = ConvNet.ResNet50()

feature_extractor.summary()

2022-07-25 21:28:47.600521: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-25 21:28:51.858565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-07-25 21:28:51.862218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-07-25 21:28:51.864229: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
#try getting features and time
with tf.device('/CPU:0'):
    features = np.empty((N, 461, 2048), dtype=np.float32)
    start = time.time()

    for i, video in enumerate(videos):
        if i % 50 == 0:
            print(f"Video {i}...")
        features[i, ...] = feature_extractor.predict_on_batch(video)

    stop = time.time()
    print(f"Finished extracting features from all {len(videos)} videos in {stop-start} seconds.")
    features.shape

Video 0...
Video 50...
Video 100...
Video 150...
Video 200...
Video 250...
Video 300...
Video 350...
Finished extracting features from all 364 videos in 3518.14151930809 seconds.


## Split Data

In [9]:
features.shape

(364, 461, 2048)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

train_index = list(X_train.index)
test_index = list(X_test.index)

# index data accordingly
train_features, train_labels = features[train_index], labels[train_index]
test_features, test_labels = features[test_index], labels[test_index]

# reshape label arrays as horizontal arrays
train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

In [11]:
print(train_features.shape)
print(test_features.shape)

(291, 461, 2048)
(73, 461, 2048)


## Train RNN Model

In [12]:
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [13]:
# recurrent_dropout does not allow training to use cuDNN kernel
features_input       = keras.Input((461, 2048))
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True, recurrent_dropout=0.5))(features_input)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(x)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True, recurrent_dropout=0.5))(x)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32))(x)

output               = keras.layers.Dense(2, activation="softmax")(x) #2 bc 2 class categories (0,1)
model                = keras.Model(features_input, output)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

my_callbacks    = [keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                 patience=3,
                                                 mode="max",
                                                 min_delta = 0.01,
                                                 restore_best_weights=True)]
history = model.fit(train_features, 
                    train_labels,
                    validation_split = 0.2,
                    epochs = 15,
                    callbacks = my_callbacks,
                    verbose= 1)

print('Done training.')

loss, accuracy = model.evaluate(test_features, test_labels)
print(f"Test Metrics - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/15


2022-07-25 22:59:09.930752: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400


8/8 [==============================] - 82s 8s/step - loss: 0.5907 - accuracy: 0.7069 - val_loss: 0.4716 - val_accuracy: 0.8136
Epoch 2/15
8/8 [==============================] - 58s 7s/step - loss: 0.4586 - accuracy: 0.7845 - val_loss: 0.4937 - val_accuracy: 0.7627
Epoch 3/15
8/8 [==============================] - 57s 7s/step - loss: 0.3787 - accuracy: 0.8362 - val_loss: 0.4833 - val_accuracy: 0.7627
Epoch 4/15
8/8 [==============================] - 57s 7s/step - loss: 0.3282 - accuracy: 0.8664 - val_loss: 0.4261 - val_accuracy: 0.8305
Epoch 5/15
8/8 [==============================] - 57s 7s/step - loss: 0.2203 - accuracy: 0.9181 - val_loss: 0.4987 - val_accuracy: 0.7458
Epoch 6/15
8/8 [==============================] - 56s 7s/step - loss: 0.1490 - accuracy: 0.9483 - val_loss: 0.5640 - val_accuracy: 0.7288
Epoch 7/15
8/8 [==============================] - 56s 7s/step - loss: 0.0789 - accuracy: 0.9784 - val_loss: 0.6080 - val_accuracy: 0.7627
Done training.
3/3 [=========================

In [15]:
#remove recurrent_dropout to train model using cuDNN kernel

features_input       = keras.Input((461, 2048))
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(features_input)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(x)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(x)
x                    = keras.layers.Bidirectional(keras.layers.LSTM(32))(x)

output               = keras.layers.Dense(2, activation="softmax")(x) #2 bc 2 class categories (0,1)
model                = keras.Model(features_input, output)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

my_callbacks    = [keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                 patience=3,
                                                 mode="max",
                                                 min_delta = 0.01,
                                                 restore_best_weights=True)]
history = model.fit(train_features, 
                    train_labels,
                    validation_split = 0.2,
                    epochs = 15,
                    callbacks = my_callbacks,
                    verbose= 1)

print('Done training.')

loss, accuracy = model.evaluate(test_features, test_labels)
print(f"Test Metrics - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/15
8/8 [==============================] - 23s 845ms/step - loss: 0.6255 - accuracy: 0.6724 - val_loss: 0.5256 - val_accuracy: 0.7627
Epoch 2/15
8/8 [==============================] - 2s 195ms/step - loss: 0.4842 - accuracy: 0.7931 - val_loss: 0.4484 - val_accuracy: 0.7966
Epoch 3/15
8/8 [==============================] - 1s 193ms/step - loss: 0.3695 - accuracy: 0.8448 - val_loss: 0.4288 - val_accuracy: 0.7797
Epoch 4/15
8/8 [==============================] - 1s 189ms/step - loss: 0.2562 - accuracy: 0.9052 - val_loss: 0.4494 - val_accuracy: 0.7966
Epoch 5/15
8/8 [==============================] - 2s 194ms/step - loss: 0.1696 - accuracy: 0.9310 - val_loss: 0.6314 - val_accuracy: 0.7458
Done training.
3/3 [==============================] - 0s 67ms/step - loss: 0.4941 - accuracy: 0.7671
Test Metrics - Loss: 0.494140625, Accuracy: 0.767123281955719


## Trying to Distribute Data + Run RNN Model

In [19]:
strategy = tf.distribute.MirroredStrategy()

global_batch_size = 91*4
dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).batch(global_batch_size)
dist_dataset = strategy.experimental_distribute_dataset(dataset)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2022-07-25 23:17:29.071859: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 291
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:1273"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 461
        }
        dim {
          size: 2048
        }
      }
      shape {
        dim {
          size: 1
        }
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_P

In [27]:
with strategy.scope():
    features_input       = keras.Input((461, 2048))
    x                    = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(features_input)
    x                    = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    x                    = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    x                    = keras.layers.Bidirectional(keras.layers.LSTM(128))(x)

    output               = keras.layers.Dense(2, activation="softmax")(x) #2 bc 2 class categories (0,1)
    model                = keras.Model(features_input, output)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(dist_dataset,
                    epochs = 15,
                    steps_per_epoch = 4,
                    verbose= 1)

print('Done training.')

loss, accuracy = model.evaluate(test_features, test_labels)
print(f"Test Metrics - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/15
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
4/4 [==============================] - 74s 28ms/step - loss: 0.6833 - accuracy: 0.5945
Done training.


2022-07-25 23:24:30.438061: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_320351"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\023FlatMapDataset:1348"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PR

2/3 [===================>..........] - ETA: 0s - loss: 0.6597 - accuracy: 0.6562 

2022-07-25 23:24:48.150847: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1563 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/stream_executor/cuda/cuda_dnn.cc(1788): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
2022-07-25 23:24:48.150918: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1563 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/stream_executor/cuda/cuda_dnn.cc(1788): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'


CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node div_no_nan_1/ReadVariableOp_6/_140}}]] [Op:__inference_test_function_334945]

In [14]:
#log training and validation metrics on wandb
for epoch, train_loss in enumerate(history.history['loss']):
    wandb.log({'training_loss': train_loss, "epoch": epoch})
    
for epoch, train_acc in enumerate(history.history['accuracy']):
    wandb.log({'training_accuracy': train_acc, "epoch": epoch})
    
for epoch, val_loss in enumerate(history.history['val_loss']):
    wandb.log({'val_loss': val_loss, "epoch": epoch})
    
for epoch, val_acc in enumerate(history.history['val_accuracy']):
    wandb.log({'val_accuracy': val_acc, "epoch": epoch})
    
print('Done Logging WandB metrics.')

Done Logging WandB metrics.


In [28]:
wandb.finish()

epoch,▁▂▃▅▆▇█▁▂▃▅▆▇█▁▂▃▅▆▇█▁▂▃▅▆▇█
training_accuracy,▁▃▄▅▆▇█
training_loss,█▆▅▄▃▂▁
val_accuracy,▇▃▃█▂▁▃
val_loss,▃▄▃▁▄▆█
epoch,6
training_accuracy,0.97845
training_loss,0.07892
val_accuracy,0.76271
val_loss,0.60799
